In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "d5f8fc34",
   "metadata": {},
   "outputs": [],
   "source": [
    "import os\n",
    "from cryptography.hazmat.primitives import hashes, serialization\n",
    "from cryptography.hazmat.primitives.asymmetric import rsa, padding\n",
    "from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC\n",
    "from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes\n",
    "from cryptography.hazmat.backends import default_backend\n",
    "from cryptography.hazmat.primitives import padding as sym_padding\n",
    "from time import time"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "37ecd040",
   "metadata": {},
   "outputs": [],
   "source": [
    "def save_to_file(filename, data):\n",
    "    with open(filename, 'wb') as f:\n",
    "        f.write(data)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "a29928ee",
   "metadata": {},
   "outputs": [],
   "source": [
    "def load_from_file(filename):\n",
    "    with open(filename, 'rb') as f:\n",
    "        return f.read()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "id": "b6ce1459",
   "metadata": {},
   "outputs": [],
   "source": [
    "def generate_aes_key(length):\n",
    "    key = os.urandom(length)\n",
    "    save_to_file(f'aes_key_{length * 8}.key', key)\n",
    "    return key"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "id": "82676f87",
   "metadata": {},
   "outputs": [],
   "source": [
    "def aes_encrypt(data, key, mode):\n",
    "    iv = os.urandom(16)\n",
    "    if mode == 'ECB':\n",
    "        cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())\n",
    "    elif mode == 'CFB':\n",
    "        cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())\n",
    "    encryptor = cipher.encryptor()\n",
    "    padder = sym_padding.PKCS7(128).padder()\n",
    "    padded_data = padder.update(data) + padder.finalize()\n",
    "    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()\n",
    "    save_to_file('aes_encrypted.bin', iv + encrypted_data)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "id": "f40f1581",
   "metadata": {},
   "outputs": [],
   "source": [
    "def aes_decrypt(key, mode):\n",
    "    encrypted_data = load_from_file('aes_encrypted.bin')\n",
    "    iv, encrypted_data = encrypted_data[:16], encrypted_data[16:]\n",
    "    if mode == 'ECB':\n",
    "        cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())\n",
    "    elif mode == 'CFB':\n",
    "        cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())\n",
    "    decryptor = cipher.decryptor()\n",
    "    decrypted_data = decryptor.update(encrypted_data) + decryptor.finalize()\n",
    "    unpadder = sym_padding.PKCS7(128).unpadder()\n",
    "    data = unpadder.update(decrypted_data) + unpadder.finalize()\n",
    "    print(data)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "id": "247ef4cf",
   "metadata": {},
   "outputs": [],
   "source": [
    "def generate_rsa_keys():\n",
    "    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend())\n",
    "    public_key = private_key.public_key()\n",
    "    save_to_file('rsa_private_key.pem', private_key.private_bytes(\n",
    "        encoding=serialization.Encoding.PEM,\n",
    "        format=serialization.PrivateFormat.TraditionalOpenSSL,\n",
    "        encryption_algorithm=serialization.NoEncryption()\n",
    "    ))\n",
    "    save_to_file('rsa_public_key.pem', public_key.public_bytes(\n",
    "        encoding=serialization.Encoding.PEM,\n",
    "        format=serialization.PublicFormat.SubjectPublicKeyInfo\n",
    "    ))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "id": "fee33757",
   "metadata": {},
   "outputs": [],
   "source": [
    "def rsa_encrypt(data, public_key):\n",
    "    encrypted_data = public_key.encrypt(\n",
    "        data,\n",
    "        padding.OAEP(\n",
    "            mgf=padding.MGF1(algorithm=hashes.SHA256()),\n",
    "            algorithm=hashes.SHA256(),\n",
    "            label=None\n",
    "        )\n",
    "    )\n",
    "    save_to_file('rsa_encrypted.bin', encrypted_data)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "id": "6acaaa7e",
   "metadata": {},
   "outputs": [],
   "source": [
    "def rsa_decrypt(private_key):\n",
    "    encrypted_data = load_from_file('rsa_encrypted.bin')\n",
    "    decrypted_data = private_key.decrypt(\n",
    "        encrypted_data,\n",
    "        padding.OAEP(\n",
    "            mgf=padding.MGF1(algorithm=hashes.SHA256()),\n",
    "            algorithm=hashes.SHA256(),\n",
    "            label=None\n",
    "        )\n",
    "    )\n",
    "    print(decrypted_data)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "id": "91729294",
   "metadata": {},
   "outputs": [],
   "source": [
    "def rsa_sign(data, private_key):\n",
    "    signature = private_key.sign(\n",
    "        data,\n",
    "        padding.PSS(\n",
    "            mgf=padding.MGF1(hashes.SHA256()),\n",
    "            salt_length=padding.PSS.MAX_LENGTH\n",
    "        ),\n",
    "        hashes.SHA256()\n",
    "    )\n",
    "    save_to_file('rsa_signature.sig', signature)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "id": "791fda51",
   "metadata": {},
   "outputs": [],
   "source": [
    "def rsa_verify(data, signature, public_key):\n",
    "    try:\n",
    "        public_key.verify(\n",
    "            signature,\n",
    "            data,\n",
    "            padding.PSS(\n",
    "                mgf=padding.MGF1(hashes.SHA256()),\n",
    "                salt_length=padding.PSS.MAX_LENGTH\n",
    "            ),\n",
    "            hashes.SHA256()\n",
    "        )\n",
    "        print(\"Verification successful\")\n",
    "    except Exception as e:\n",
    "        print(f\"Verification failed: {e}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "id": "97a3d5ed",
   "metadata": {},
   "outputs": [],
   "source": [
    "def sha256_hash(data):\n",
    "    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())\n",
    "    digest.update(data)\n",
    "    hash_value = digest.finalize()\n",
    "    print(hash_value.hex())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "id": "6ccce7d5",
   "metadata": {},
   "outputs": [],
   "source": [
    "def main():\n",
    "    while True:\n",
    "        print(\"Select operation:\")\n",
    "        print(\"1. AES Encryption\")\n",
    "        print(\"2. AES Decryption\")\n",
    "        print(\"3. RSA Encryption\")\n",
    "        print(\"4. RSA Decryption\")\n",
    "        print(\"5. RSA Signature\")\n",
    "        print(\"6. RSA Verification\")\n",
    "        print(\"7. SHA-256 Hashing\")\n",
    "        print(\"8. Exit\")\n",
    "        choice = int(input(\"Enter choice: \"))\n",
    "        if choice == 1:\n",
    "            key_length = int(input(\"Enter AES key length (128 or 256): \"))\n",
    "            mode = input(\"Enter AES mode (ECB or CFB): \")\n",
    "            key = generate_aes_key(key_length // 8)\n",
    "            data = input(\"Enter data to encrypt: \").encode()\n",
    "            start = time()\n",
    "            aes_encrypt(data, key, mode)\n",
    "            end = time()\n",
    "            print(f\"AES encryption took {end - start} seconds\")\n",
    "        elif choice == 2:\n",
    "            key_length = int(input(\"Enter AES key length (128 or 256): \"))\n",
    "            mode = input(\"Enter AES mode (ECB or CFB): \")\n",
    "            key = load_from_file(f'aes_key_{key_length}.key')\n",
    "            start = time()\n",
    "            aes_decrypt(key, mode)\n",
    "            end = time()\n",
    "            print(f\"AES decryption took {end - start} seconds\")\n",
    "        elif choice == 3:\n",
    "            generate_rsa_keys()\n",
    "            data = input(\"Enter data to encrypt: \").encode()\n",
    "            public_key = serialization.load_pem_public_key(load_from_file('rsa_public_key.pem'), backend=default_backend())\n",
    "            start = time()\n",
    "            rsa_encrypt(data, public_key)\n",
    "            end = time()\n",
    "            print(f\"RSA encryption took {end - start} seconds\")\n",
    "        elif choice == 4:\n",
    "            private_key = serialization.load_pem_private_key(load_from_file('rsa_private_key.pem'), password=None, backend=default_backend())\n",
    "            start = time()\n",
    "            rsa_decrypt(private_key)\n",
    "            end = time()\n",
    "            print(f\"RSA decryption took {end - start} seconds\")\n",
    "        elif choice == 5:\n",
    "            private_key = serialization.load_pem_private_key(load_from_file('rsa_private_key.pem'), password=None, backend=default_backend())\n",
    "            data = input(\"Enter data to sign: \").encode()\n",
    "            start = time()\n",
    "            rsa_sign(data, private_key)\n",
    "            end = time()\n",
    "            print(f\"RSA signature took {end - start} seconds\")\n",
    "        elif choice == 6:\n",
    "            public_key = serialization.load_pem_public_key(load_from_file('rsa_public_key.pem'), backend=default_backend())\n",
    "            data = input(\"Enter data to verify: \").encode()\n",
    "            signature = load_from_file('rsa_signature.sig')\n",
    "            start = time()\n",
    "            rsa_verify(data, signature, public_key)\n",
    "            end = time()\n",
    "            print(f\"RSA verification took {end - start} seconds\")\n",
    "        elif choice == 7:\n",
    "            data = input(\"Enter data to hash: \").encode()\n",
    "            start = time()\n",
    "            sha256_hash(data)\n",
    "            end = time()\n",
    "            print(f\"SHA-256 hashing took {end - start} seconds\")\n",
    "        elif choice == 8:\n",
    "            break\n",
    "        else:\n",
    "            print(\"Invalid choice, please try again\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "id": "6f7521bd",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Select operation:\n",
      "1. AES Encryption\n",
      "2. AES Decryption\n",
      "3. RSA Encryption\n",
      "4. RSA Decryption\n",
      "5. RSA Signature\n",
      "6. RSA Verification\n",
      "7. SHA-256 Hashing\n",
      "8. Exit\n",
      "AES encryption took 0.025913238525390625 seconds\n",
      "Select operation:\n",
      "1. AES Encryption\n",
      "2. AES Decryption\n",
      "3. RSA Encryption\n",
      "4. RSA Decryption\n",
      "5. RSA Signature\n",
      "6. RSA Verification\n",
      "7. SHA-256 Hashing\n",
      "8. Exit\n",
      "b'Deep Learning is fascinating!'\n",
      "AES decryption took 0.0002543926239013672 seconds\n",
      "Select operation:\n",
      "1. AES Encryption\n",
      "2. AES Decryption\n",
      "3. RSA Encryption\n",
      "4. RSA Decryption\n",
      "5. RSA Signature\n",
      "6. RSA Verification\n",
      "7. SHA-256 Hashing\n",
      "8. Exit\n",
      "RSA encryption took 0.00038123130798339844 seconds\n",
      "Select operation:\n",
      "1. AES Encryption\n",
      "2. AES Decryption\n",
      "3. RSA Encryption\n",
      "4. RSA Decryption\n",
      "5. RSA Signature\n",
      "6. RSA Verification\n",
      "7. SHA-256 Hashing\n",
      "8. Exit\n",
      "b'RSA is a public key cryptography system.'\n",
      "RSA decryption took 0.0011818408966064453 seconds\n",
      "Select operation:\n",
      "1. AES Encryption\n",
      "2. AES Decryption\n",
      "3. RSA Encryption\n",
      "4. RSA Decryption\n",
      "5. RSA Signature\n",
      "6. RSA Verification\n",
      "7. SHA-256 Hashing\n",
      "8. Exit\n",
      "RSA signature took 0.001916646957397461 seconds\n",
      "Select operation:\n",
      "1. AES Encryption\n",
      "2. AES Decryption\n",
      "3. RSA Encryption\n",
      "4. RSA Decryption\n",
      "5. RSA Signature\n",
      "6. RSA Verification\n",
      "7. SHA-256 Hashing\n",
      "8. Exit\n",
      "Verification successful\n",
      "RSA verification took 0.00031304359436035156 seconds\n",
      "Select operation:\n",
      "1. AES Encryption\n",
      "2. AES Decryption\n",
      "3. RSA Encryption\n",
      "4. RSA Decryption\n",
      "5. RSA Signature\n",
      "6. RSA Verification\n",
      "7. SHA-256 Hashing\n",
      "8. Exit\n",
      "Verification failed: \n",
      "RSA verification took 0.00025200843811035156 seconds\n",
      "Select operation:\n",
      "1. AES Encryption\n",
      "2. AES Decryption\n",
      "3. RSA Encryption\n",
      "4. RSA Decryption\n",
      "5. RSA Signature\n",
      "6. RSA Verification\n",
      "7. SHA-256 Hashing\n",
      "8. Exit\n",
      "d018bb6218aa8aff8eef6a4a4423f88102a3b5871f75cd2176323a85773dfda8\n",
      "SHA-256 hashing took 0.00012612342834472656 seconds\n",
      "Select operation:\n",
      "1. AES Encryption\n",
      "2. AES Decryption\n",
      "3. RSA Encryption\n",
      "4. RSA Decryption\n",
      "5. RSA Signature\n",
      "6. RSA Verification\n",
      "7. SHA-256 Hashing\n",
      "8. Exit\n"
     ]
    }
   ],
   "source": [
    "if __name__ == \"__main__\":\n",
    "    main()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "99c72d37",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.12.3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}